# **Preparations**

### **Importing necessary modules**

In [ ]:
import psycopg2
import os

### **Establishing connection and declaring the cursor**

In [ ]:
def connection():
    conn = psycopg2.connect(
        dbname= 'projeto1',
        user= 'postgres',
        password= '',   # Replace with actual password
        host= 'localhost',
        port= '5432'
    )
    cursor = conn.cursor()
    return conn, cursor

### **Creating functions that execute queries**

In [ ]:
def run_and_save_sql(cursor, query, file):
    output = f'copy ({query}) to stdout with csv header'
    cursor.copy_expert(output, file)

In [ ]:
def run_sql(cursor, query):
    cursor.execute(query)
    column_names = [desc[0] for desc in cursor.description]
    return column_names, cursor.fetchall()
    

# **Declaring queries**

### **Get mortality rate by state**

In [ ]:
mortality_by_state_sql = """
    -- Estimate population, total deaths, and mortality rate (per 1000) by state
    WITH population_by_state AS (
        SELECT 
            s."name" AS state_name,
            SUM(m."population_estimate_2009") AS estimated_population
        FROM "Municipality" m
        JOIN "State" s ON s."IBGE_state_code" = m."IBGE_state_code"
        GROUP BY s."name"
    ),

    deaths_by_state AS (
        SELECT 
            s."name" AS state_name,
            COUNT(*) AS total_deaths             
        FROM "Death" d
        JOIN "Municipality" m ON d."location_code" = m."IBGE_city_code"
        JOIN "State" s ON s."IBGE_state_code" = m."IBGE_state_code"
        GROUP BY s."name"
    )

    SELECT 
        pop.state_name AS "State",
        pop.estimated_population AS "Estimated_Population",
        dth.total_deaths AS "Total_Deaths",
        ROUND(
            CAST((1000 * dth.total_deaths / pop.estimated_population) AS NUMERIC),
            4
        ) AS "Mortality_Rate"
    FROM population_by_state pop
    JOIN deaths_by_state dth ON pop.state_name = dth.state_name
    ORDER BY "Mortality_Rate" DESC
    """

### **Get the most common cause of death by age**

In [ ]:
death_cause_by_age_sql = """
    -- Find the most common cause of death (modal cause) for each age
    WITH causes_by_age AS (
        SELECT  
            dth."basic_cause_id" AS cause_id,
            ROUND(CAST(dcs."age" AS NUMERIC), 0) AS age,
            COUNT(*) AS death_count
        FROM "Death" dth
        JOIN "Deceased" dcs ON dth."death_id" = dcs."deceased_id"
        GROUP BY age, dth."basic_cause_id"
    ),

    age_totals_and_max AS (
        SELECT 
            age,
            MAX(death_count) AS max_death_count,
            SUM(death_count) AS total_deaths_for_age
        FROM causes_by_age
        GROUP BY age
    )

    SELECT  
        cba.age,
        dc."description" AS most_common_cause,
        cba.death_count AS deaths_from_cause,
        atm.total_deaths_for_age,
        ROUND(100.0 * cba.death_count / atm.total_deaths_for_age, 4) AS cause_percentage
    FROM causes_by_age cba
    JOIN age_totals_and_max atm 
        ON cba.age = atm.age AND cba.death_count = atm.max_death_count
    JOIN "DeathCause" dc 
        ON dc."basic_cause_id" = cba.cause_id
    ORDER BY cba.age
    """

### **Get the average age at death (proxy for life expectancy) by city GDP per capita decile**

In [ ]:
life_expectancy_by_gdp_sql = """
    -- Analyze death statistics by GDP per capita decile
	WITH city_deciles AS (
		SELECT 
			"IBGE_city_code" AS city_code,
			NTILE(10) OVER (ORDER BY "GDP_per_capita") AS decile
		FROM "Municipality"
	),

	city_stats_by_decile AS (
		SELECT 
			decile,
			COUNT(*) AS number_of_cities,
			MIN(m."GDP_per_capita") AS lowest_gdp_per_capita,
			MAX(m."GDP_per_capita") AS highest_gdp_per_capita,
			SUM(m."population_estimate_2009") AS total_population
		FROM "Municipality" m
		JOIN city_deciles d ON m."IBGE_city_code" = d.city_code
		GROUP BY decile
	),

	death_stats_by_decile AS (
		SELECT 
			d.decile,
			COUNT(*) AS total_deaths,
			ROUND(CAST(AVG(dc."age") AS NUMERIC), 4) AS mean_age_of_death,
			PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY dc."age") AS median_age_of_death
		FROM "Death" dt
		JOIN "Deceased" dc ON dt."death_id" = dc."deceased_id"
		JOIN city_deciles d ON dt."location_code" = d.city_code
		GROUP BY d.decile
	)

	SELECT 
		cs.decile AS decile,
		cs.number_of_cities,
		cs.lowest_gdp_per_capita,
		cs.highest_gdp_per_capita,
		cs.total_population,
		ds.total_deaths,
		ds.mean_age_of_death,
		ds.median_age_of_death
	FROM city_stats_by_decile cs
	JOIN death_stats_by_decile ds ON cs.decile = ds.decile
	ORDER BY cs.decile
    """

### **Get the top 3 causes of deaths for children under five, grouped by mother's education level**

In [ ]:
cause_by_mothers_education_sql = """
    -- Top 3 causes of death (under age 5) by mother's education level
	WITH deaths_by_mother_edu AS (
		SELECT  
			mom."education_level" AS education_level,
			dth."basic_cause_id" AS cause_id,
			ROUND(AVG(CAST(mom."age" AS NUMERIC)), 2) AS mothers_avg_age,
			COUNT(*) AS death_count
		FROM "Deceased" dcs
		JOIN "Mother" mom 
			ON dcs."deceased_id" = mom."mother_id"
			AND dcs."age" < 5.0
			AND mom."education_level" IN (0, 1, 2, 3, 4, 5)
		JOIN "Death" dth
			ON dcs."deceased_id" = dth."death_id"
		GROUP BY mom."education_level", dth."basic_cause_id"
	),

	ranked_causes AS (
		SELECT  
			education_level,
			cause_id,
			death_count,
			RANK() OVER (
				PARTITION BY education_level 
				ORDER BY death_count DESC
			) AS cause_rank
		FROM deaths_by_mother_edu
	),

	edu_totals AS (
		SELECT  
			education_level,
			SUM(death_count) AS total_death_count
		FROM deaths_by_mother_edu
		GROUP BY education_level
	)

	SELECT
		CASE dme.education_level
			WHEN 0 THEN 'None'
			WHEN 1 THEN 'Elementary School'
			WHEN 2 THEN 'Middle School'
			WHEN 3 THEN 'High School'
			WHEN 4 THEN 'Incomplete Higher Education'
			WHEN 5 THEN 'Complete Higher Education'
			ELSE 'Other'
		END AS mothers_education_level,
		rc.cause_rank,
		dc."description" AS cause_of_death,
		dme.mothers_avg_age,
		rc.death_count AS cause_death_count,
		et.total_death_count,
		ROUND(100.0 * rc.death_count / et.total_death_count, 2) AS cause_percentage

	FROM deaths_by_mother_edu dme
	JOIN ranked_causes rc 
		ON rc.education_level = dme.education_level 
		AND rc.cause_id = dme.cause_id 
		AND rc.cause_rank <= 3
	JOIN "DeathCause" dc 
		ON dc."basic_cause_id" = dme.cause_id
	JOIN edu_totals et 
		ON et.education_level = dme.education_level

	ORDER BY rc.education_level, rc.cause_rank
    """

### **Get the mortality rate grouped by city population per healthcare unit (inverse healthcare unit density)**

In [ ]:
mortality_by_health_unit_density_sql = """
    -- Analyze non-external mortality rates grouped by population per health unit
	WITH population_per_unit AS (
		SELECT
			m."IBGE_city_code",
			m."population_estimate_2009",
			u.unit_count,
			m."population_estimate_2009" / u.unit_count AS persons_per_unit
		FROM "Municipality" m
		JOIN (
			SELECT
				"IBGE_city_code",
				COUNT(*) AS unit_count
			FROM "BasicHealthUnit"
			GROUP BY "IBGE_city_code"
		) u ON m."IBGE_city_code" = u."IBGE_city_code"
	),

	non_external_deaths AS (
		SELECT
			m."IBGE_city_code",
			m."population_estimate_2009",
			d.total_deaths AS non_external_death_count
		FROM "Municipality" m
		JOIN (
			SELECT
				"location_code",
				COUNT(*) AS total_deaths
			FROM "Death"
			WHERE NOT (
				"basic_cause_id" LIKE 'S%' OR
				"basic_cause_id" LIKE 'T%' OR
				"basic_cause_id" LIKE 'V%' OR
				"basic_cause_id" LIKE 'W%' OR
				"basic_cause_id" LIKE 'X%' OR
				"basic_cause_id" LIKE 'Y%'
			)
			GROUP BY "location_code"
		) d ON m."IBGE_city_code" = d."location_code"
	),

	city_bins AS (
		SELECT
			ppu."IBGE_city_code",
			ppu."population_estimate_2009",
			ppu.unit_count,
			ppu.persons_per_unit,
			nd.non_external_death_count,
			CASE 
				WHEN ppu.persons_per_unit < 1500 THEN 1
				WHEN ppu.persons_per_unit < 2000 THEN 2
				WHEN ppu.persons_per_unit < 2500 THEN 3
				WHEN ppu.persons_per_unit < 3000 THEN 4
				WHEN ppu.persons_per_unit < 4000 THEN 5
				WHEN ppu.persons_per_unit < 5000 THEN 6
				ELSE 7
			END AS ppu_range
		FROM population_per_unit ppu
		JOIN non_external_deaths nd ON ppu."IBGE_city_code" = nd."IBGE_city_code"
	)

	SELECT
		CASE ppu_range
			WHEN 1 THEN 'Fewer than 1500'
			WHEN 2 THEN '1500–1999'
			WHEN 3 THEN '2000–2499'
			WHEN 4 THEN '2500–2999'
			WHEN 5 THEN '3000–3999'
			WHEN 6 THEN '4000–4999'
			WHEN 7 THEN '5000 or more'
		END AS individuals_per_health_unit,
		COUNT(*) AS number_of_cities,
		SUM(population_estimate_2009) AS total_population,
		SUM(unit_count) AS total_units,
		SUM(non_external_death_count) AS total_non_external_deaths,
		ROUND(
			CAST(1000 * SUM(non_external_death_count) / SUM(population_estimate_2009) AS NUMERIC),
			4
		) AS mortality_rate
	FROM city_bins
	GROUP BY ppu_range
	ORDER BY ppu_range
    """

# **Executing queries and saving results as csv**

In [ ]:
conn,cursor = connection()
with open(os.path.abspath('../results/mortality_by_state.csv'), "w") as f:
    run_and_save_sql(cursor, mortality_by_state_sql, f)
cursor.close()
conn.close()

In [ ]:
conn,cursor = connection()
with open(os.path.abspath('../results/death_cause_by_age.csv'), "w") as f:
    run_and_save_sql(cursor, death_cause_by_age_sql, f)
cursor.close()
conn.close()

In [ ]:
conn,cursor = connection()
with open(os.path.abspath('../results/life_expectancy_by_gdp.csv'), "w") as f:
    run_and_save_sql(cursor, life_expectancy_by_gdp_sql, f)
cursor.close()
conn.close()

In [ ]:
conn,cursor = connection()
with open(os.path.abspath('../results/cause_by_mothers_education.csv'), "w") as f:
    run_and_save_sql(cursor, cause_by_mothers_education_sql, f)
cursor.close()
conn.close()

In [ ]:
conn,cursor = connection()
with open(os.path.abspath('../results/mortality_by_health_unit_density.csv'), "w") as f:
    run_and_save_sql(cursor, mortality_by_health_unit_density_sql, f)
cursor.close()
conn.close()

# **Executing queries and printing results**

In [ ]:
queries = {1 : mortality_by_state_sql,
           2 : death_cause_by_age_sql,
           3 : life_expectancy_by_gdp_sql,
           4 : cause_by_mothers_education_sql,
           5 : mortality_by_health_unit_density_sql}
conn,cursor = connection()
col_names, rows = run_sql(cursor, queries[5])   # change index to desired query
cursor.close()
conn.close()

for col in col_names:
    print(f'| {col} ', end='')
print('|')
for row in rows:
    for value in row:
        print(f'| {value} ', end='')
    print('|')